In [124]:
!pip install langchain_community
!pip install pymupdf
!pip install -qU pypdf
!pip install -qU langchain-text-splitters
!pip install -qU langchain-text-splitters FlagEmbedding faiss-cpu

In [130]:
!python -m pip show FlagEmbedding

Name: FlagEmbedding
Version: 1.3.5
Summary: FlagEmbedding
Home-page: https://github.com/FlagOpen/FlagEmbedding
Author: 
Author-email: 2906698981@qq.com
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: accelerate, datasets, ir-datasets, peft, protobuf, sentence-transformers, sentencepiece, torch, transformers
Required-by: 


In [131]:
!python -m pip install -U "FlagEmbedding>=1.2.7"

In [133]:
from langchain_community.vectorstores import FAISS

In [ ]:
!ollama run llama3

^C


In [135]:
!nohup ollama serve > /content/ollama.log 2>&1 &

In [136]:
!ollama pull llama3

In [ ]:
!curl http://localhost:11434/api/tags

{"models":[{"name":"llama3:latest","model":"llama3:latest","modified_at":"2025-06-02T03:33:08.863559397Z","size":4661224676,"digest":"365c0bd3c000a25d28ddbf732fe1c6add414de7275464c4e4d1c3b5fcb5d8ad1","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8.0B","quantization_level":"Q4_0"}}]}

In [ ]:
# 1) 경로·파라미터 정의(완료)

PDF_PATH   = "2022개정교육과정자유학기운영안내서.pdf"   # ← 처리할 PDF 경로
INDEX_DIR  = "faiss_bge_m3_index"                       # 인덱스 저장 폴더
CHUNK_SIZE = 1500                                       # 청크 길이(문자)
CHUNK_OVER = 250

In [ ]:
# 파일 추가(완료)

PDF_PATH = r"/content/2025년학생안전종합계획(발송용).pdf"   # ← 처리할 PDF 경로
INDEX_DIR  = "faiss_bge_m3_index"                       # 인덱스 저장 폴더
CHUNK_SIZE = 1500                                       # 청크 길이(문자)
CHUNK_OVER = 250

In [ ]:
# 파일 추가(완료)

PDF_PATH = r"/content/2025자유학기운영참고자료.pdf"   # ← 처리할 PDF 경로
INDEX_DIR  = "faiss_bge_m3_index"                       # 인덱스 저장 폴더
CHUNK_SIZE = 1500                                       # 청크 길이(문자)
CHUNK_OVER = 250

In [137]:
# 파일 추가

PDF_PATH = r"/content/2025중등교원인사업무처리요령1권.pdf"   # ← 처리할 PDF 경로
INDEX_DIR  = "faiss_bge_m3_index"                       # 인덱스 저장 폴더
CHUNK_SIZE = 1500                                       # 청크 길이(문자)
CHUNK_OVER = 250

In [138]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
🔹 BGE-M3 + FAISS 누적 인덱싱 스크립트
   · PDF 하나씩 처리할 때마다 같은 인덱스 폴더 안으로 계속 벡터를 추가합니다.
   · 인덱스 폴더가 없으면 새로 만들고, 있으면 자동으로 로드하여 add_documents().
   · 임베딩-LLM 일체 API 호출 없이 전부 로컬에서 구동됩니다.
"""

# 0) 필수 패키지 (첫 실행 시 한 번만 설치)
#    ── 주피터 노트북이면 !pip 로, 로컬 쉘이면 pip install …
# pip install -qU langchain-community langchain-text-splitters \
#                faiss-cpu FlagEmbedding pymupdf


# 2) 모듈 import
import numpy as np
from pathlib import Path
from FlagEmbedding import FlagModel
from langchain.embeddings.base import Embeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# 3) BGE-M3 임베딩 래퍼 (LangChain 호환)
class BGEEmbedding(Embeddings):
    """FlagEmbedding → LangChain 래퍼 (float32 + L2 정규화)"""

    def __init__(self, model_name="BAAI/bge-m3", fp16=True):
        self.model = FlagModel(model_name, use_fp16=fp16)

    def _encode(self, texts):
        vecs = self.model.encode(texts, batch_size=32, max_length=8192)
        vecs = np.asarray(vecs, dtype="float32")
        vecs /= np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-12
        return vecs

    def embed_documents(self, texts):
        return self._encode(texts)

    def embed_query(self, text):
        return self._encode([text])[0]

emb_wrapper = BGEEmbedding(fp16=True)  # GPU 없으면 fp16=False

# 4) PDF → 문서 로드 & 청크 분할
loader = PyMuPDFLoader(PDF_PATH)
docs = loader.load()  # 페이지 단위 LangChain Document 리스트

splitter = RecursiveCharacterTextSplitter(
               chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVER
           )
chunks = splitter.split_documents(docs)
print(f"📄  {PDF_PATH} → 분할 청크 {len(chunks):,}개")


# 5) FAISS 인덱스: 로드 후 add  ⟷  최초 생성
if Path(INDEX_DIR).exists():
    print("🔄 기존 인덱스 로드 중…")
    db = FAISS.load_local(
        INDEX_DIR,
        emb_wrapper,
        allow_dangerous_deserialization=True
    )
    print(f"현재 벡터 수: {db.index.ntotal:,}")

    print("새 문서를 인덱스에 추가 중…")
    db.add_documents(chunks)
else:
    print("인덱스 최초 생성 중…")
    db = FAISS.from_documents(chunks, emb_wrapper)

# 저장
db.save_local(INDEX_DIR)
print(f"저장 완료!  총 벡터 {db.index.ntotal:,}개 → {INDEX_DIR}/")

📄  /content/2025중등교원인사업무처리요령1권.pdf → 분할 청크 434개
🔄 기존 인덱스 로드 중…
현재 벡터 수: 668
새 문서를 인덱스에 추가 중…


pre tokenize: 100%|██████████| 14/14 [00:02<00:00,  6.74it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 14/14 [28:34<00:00, 122.45s/it]

저장 완료!  총 벡터 1,102개 → faiss_bge_m3_index/


In [123]:
if __name__ == "__main__":
    query = "학교안전사고 예방 ·관리 체계의 고도화 과제에 대한 담당부서는 어디고 연락처가 어떻게 돼?"
    hits  = db.similarity_search(query, k=3)
    print("\nTop-3 유사 청크")
    for d in hits:
        pg = d.metadata.get("page_number", "?")
        print(f"(p.{pg}) {d.page_content[:120]} …")


Top-3 유사 청크
(p.?) - 12 -
1
학교안전 관리 체계
1-1. 학교안전사고 예방·관리 체계
1-1-1. 학교안전사고 예방·관리 체계의 고도화
담당부서
학교안전과
담당자
장학사전청
☎239-0841
□ 개요
  ❍ 학교안전사고를 예방하 …
(p.?) - 12 -
1
학교안전 관리 체계
1-1. 학교안전사고 예방·관리 체계
1-1-1. 학교안전사고 예방·관리 체계의 고도화
담당부서
학교안전과
담당자
장학사전청
☎239-0841
□ 개요
  ❍ 학교안전사고를 예방하 …
(p.?) - 9 -
2
주요 추진 과제
□ 추진과제 목록
추  진  과  제
담당부서
연락처페이지
1. 학교안전 관리 체계
1-1. 학교안전사고 예방·관리 체계
1-1-1. 학교안전사고예방·관리체계의고도화학교안전과239-08 …
